In [1]:
!pip install pandas



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
df = pd.read_csv('./DataSet/IMDB_Dataset.csv')
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df['review'][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

## 1. Make the columns into lower case

In [9]:
df['review'][3].lower()

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [13]:
df['review'] = df['review'].str.lower()

In [14]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


## 2. Remove html tags 

In [16]:
import re

def remove_html_tags(text):
    pattern = re.compile(r'<.*?>')
    return pattern.sub('', text)


In [19]:
df['review'] = df['review'].apply(remove_html_tags)

In [21]:
df['review'][3]

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

## 3. Remove URLs

In [26]:
import re

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [27]:
df['review'] = df['review'].apply(remove_url)

In [28]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


## 4. Remove Punctuations

In [78]:
import string
import time

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [79]:
exclude = string.punctuation

### One way -> Best for Small DataSet

In [80]:
def remove_punc(text):
    for char in exclude:
        text = text.replace(char, '')
    return text

In [81]:
text = 'string. With. Punctuation?'

In [82]:
start = time.time()
print(remove_punc(text))
time1 = time.time() - start
print(time1)

string With Punctuation
0.00010800361633300781


### Second way -> Best for Big DataSet

In [83]:
def remove_punc1(text):
    return text.translate(str.maketrans('', '', exclude))


In [84]:
start = time.time()
print(remove_punc1(text))
time2 = time.time() - start
print(time1)

string With Punctuation
0.00010800361633300781


### Apply the same on twitter hatred speech dataset

In [98]:
df = pd.read_csv('./DataSet/twitter_hatred_speech.csv')

In [99]:
df.sample(5)

,id,label,tweet
9407,9408,0,#quote#quotes#love #heabreak#broken ðð¿ð...
19998,19999,0,well. i think it's just the whole damn next ch...
16334,16335,0,@user bra i be so bored i wanna go to practic...
1996,1997,0,@user one the flowers i planted as a seed blo...
27586,27587,0,i am not from nigeria.. i am just trying to he...


In [100]:
df['tweet'] = df['tweet'].apply(remove_punc1)

In [101]:
df.sample(5)

,id,label,tweet
5978,5979,0,dont cry over the past its gone dont stress yo...
29570,29571,0,coffee break today we have a session with the ...
20392,20393,0,user spend more time doing the things that ma...
18973,18974,0,moreno bull up you will dominate your bull a...
27965,27966,0,qampa with user founders bihday


## 5. Remove any slang(chat word treatment)

In [111]:
chat_words = {}
with open("./DataSet/slang.txt", "r") as f:
    for line in f:
        if "=" in line:  # avoid blank lines
            key, value = line.strip().split("=", 1)
            chat_words[key.strip()] = value.strip()

In [124]:
# Second key-value pair (as a tuple)
print(list(chat_words.items())[1])

# Just the second key
print(list(chat_words.keys())[1])

# Just the second value
print(list(chat_words.values())[1])


('ADIH', 'Another Day In Hell')
ADIH
Another Day In Hell


In [113]:
def chat_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)


In [116]:
chat_conversion('IMHO he is the best')

'In My Honest/Humble Opinion he is the best'

In [117]:
chat_conversion('FYI Delhi is the capital of India')

'For Your Information Delhi is the capital of India'